In [1]:
import re
import requests

from bs4 import BeautifulSoup

In [2]:
# Scrape a player page
url = "https://www.transfermarkt.us/erling-haaland/profil/spieler/418560"
player_id = url.split('/')[-1]

In [3]:
# Get headers from https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [4]:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
player_name = soup.select_one('h1[class="data-header__headline-wrapper"]').text.split('\n')[-1].strip()
player_number = soup.select_one('span[class="data-header__shirt-number"]').text.strip().replace('#', '')

In [6]:
# Let's use regex to find the player's contract expiry date
player_contract_expiry = re.search("Contract expires: (.*)", soup.text).group(1)

# Now let's add some more complex regex patterns to find the player's birthplace
player_birthplace = re.search("Place of birth:.*?([A-z].*?) ", soup.text, re.DOTALL).group(1).strip()

# We can reuse both of those ideas to find other player information but just replacing the name

# We might have to switch a couple of things though in the regex pattern like accounting for two words
player_agent = re.search("Agent:.*?([A-z].*?)\n", soup.text, re.DOTALL).group(1).strip()

# We can also use regex to find the player's height which has numbers a comma and letters
player_height = re.search("Height:.*?([0-9].*?)\n", soup.text, re.DOTALL).group(1).strip()


In [7]:
# Ok so now that we've got some basic regex patterns, let's move on to using api endpoints
# curlconverter.com
api_endpoints = [
    f"marketValueDevelopment/graph/{player_id}",
    f"transferHistory/list/{player_id}",
    f"player/{player_id}/performance"
]

for endpoint in api_endpoints:
    response = requests.get(
        'https://www.transfermarkt.us/ceapi/' + endpoint,
        headers=headers
    ).json()